CREATED: 2021-12-22
What % of all proteins are specific (separately species and genus)
How many specific genes are there 
  on average
  min
  max
  standard deviation
  histogram
  quartiles
For each species:
  how many genes are specific - both count and percentage
  top 10
  bottom 10

In [2]:
import json
from pprint import pprint

try:
    metadata_file = open("../data/metadata.json", "r")
except FileNotFoundError:
    print("\033[91m\n" + "No metadata file found, aborting" + "\n\033[0m")

try:
    trg_file = open("../large_data/trg.json", "r")
except FileNotFoundError:
    print("\033[91m\n" + "No trg file found, aborting" + "\n\033[0m")
metadata = json.load(metadata_file)
specific_genes_data = json.load(trg_file)

metadata_file.close()
trg_file.close()


Total number of genomes: 45555
Total number of genes: 147769736

In [3]:
total_genomes = len(metadata)
print(total_genomes)
total_genes = 0
for genome in metadata:
    total_genes += metadata[genome]["protein_count"]
print(total_genes)


45555
147769736


Total genes specific to species: 4150013
Total genes specific to genus: 2638854

In [6]:
total_genes_specific_to_species = 0
total_genes_specific_to_genus = 0
for genome in specific_genes_data:
    for gene in specific_genes_data[genome]:
        if gene[1] == 1:
            total_genes_specific_to_species += 1
        elif gene[1] == 0:
            total_genes_specific_to_genus += 1
        else:
            print("error")
print(total_genes_specific_to_species)
print(total_genes_specific_to_genus)

4150013
2638854


Average number of genes: 3243.7654703106136
Average number of species-specific genes: 91.09895730435737
Average number of genus-specific genes: 57.92676983865657

In [12]:
print(total_genes/total_genomes)
print(total_genes_specific_to_species/total_genomes)
print(total_genes_specific_to_genus/total_genomes)


3243.7654703106136
91.09895730435737
57.92676983865657
